In [ ]:
//TCP server to OF

const net = require('net');
const port = 7070;
const host = '127.0.0.1';

const server = net.createServer();
server.listen(port, host, () => {
    console.log('TCP Server is running on port ' + port + '.');
});

let sockets = [];

server.on('connection', function(sock) {
    console.log('CONNECTED: ' + sock.remoteAddress + ':' + sock.remotePort);
    sockets.push(sock);

    sock.on('data', function(data) {
        console.log('DATA ' + sock.remoteAddress + ': ' + data);
        // Write the data back to all the connected, the client will receive it as data from the server
        sockets.forEach(function(sock, index, array) {
            sock.write(sock.remoteAddress + ':' + sock.remotePort + " said " + data + '\n');
        });
    });

    // Add a 'close' event handler to this instance of socket
    sock.on('close', function(data) {
        let index = sockets.findIndex(function(o) {
            return o.remoteAddress === sock.remoteAddress && o.remotePort === sock.remotePort;
        })
        if (index !== -1) sockets.splice(index, 1);
        console.log('CLOSED: ' + sock.remoteAddress + ' ' + sock.remotePort);
    });
});

In [ ]:
//OSC Server to touchOSC

const osc = require('osc');

const OSC_PORT = 7071

let tcp_osc_port = new osc.TCPSocketPort({});
// change to remote host/port (address of phone/tablet) - set touchOSC on device to TCPserver
tcp_osc_port.open('192.168.1.39', OSC_PORT); 

tcp_osc_port.on('ready', () => {
  console.log('ready TCP');
});

// Bind to a UDP socket to listen for incoming OSC events.
let udp_osc_port = new osc.UDPPort({
    localAddress: "0.0.0.0",
    localPort: OSC_PORT
});

udp_osc_port.on("ready", () => {
    console.log('ready UDP');
});

udp_osc_port.open();

let osc_port = udp_osc_port;

const rawLoops = {};
const processedLoops = {};
let activeLoop = 'firstLoop';
let isRecording = false;
let isTouching = false;
const handlers = {};

function processLoop(loopKey) {
    processedLoops[loopKey] = rawLoops[loopKey];
}

handlers['/recording'] = (isRec) => {
    isRecording = isRec;
    if(isRecording) {
        rawLoops[activeLoop] = [{ts: Date.now(), pos: 'start'}];
    } else {
        rawLoops[activeLoop].push({ts: Date.now(), pos: 'end'});
        processLoop(activeLoop);
        // sockets.forEach(sock => sock.write( JSON.stringify(processedLoops) + '\n' ))
    }
}
handlers['/pos'] = (x, y) => {
    if(isRecording) {
        rawLoops[activeLoop].push({ts: Date.now(), pos: {x, y}});
    }
    //todo: send pos to OF for visualization of live touch point
}
handlers['/pos/touch'] = (touching) => {
    isTouching = touching;
    //todo: send isTouching to OF for visualization of live touch point
}

osc_port.on('message', (message) => {
    let {address, args} = message;
    console.log(message)
    if(handlers[address]) handlers[address](...args); 
});



EventEmitter {
  options: {
    localAddress: '0.0.0.0',
    localPort: 7071,
    remoteAddress: '127.0.0.1',
    remotePort: 57121
  },
  _events: [Object: null prototype] {
    data: [Function: bound ],
    open: [Function: bound ],
    ready: [Function (anonymous)],
    message: [Function (anonymous)]
  },
  _eventsCount: 4,
  socket: Socket {
    _events: [Object: null prototype] {
      error: [Array],
      listening: [Function: onListening]
    },
    _eventsCount: 2,
    _maxListeners: undefined,
    type: 'udp4',
    [Symbol(kCapture)]: false,
    [Symbol(async_id_symbol)]: 12,
    [Symbol(state symbol)]: {
      handle: [UDP],
      receiving: false,
      bindState: 1,
      connectState: 0,
      queue: undefined,
      reuseAddr: undefined,
      ipv6Only: undefined,
      recvBufferSize: undefined,
      sendBufferSize: undefined
    }
  }
}

ready UDP
{ address: '/recording', args: [ 1 ] }
{ address: '/pos/touch', args: [ true ] }
{ address: '/pos', args: [ 0.538375198841095, 0.47343891859054565 ] }
{ address: '/pos', args: [ 0.538375198841095, 0.8287749886512756 ] }
{ address: '/pos', args: [ 0.5381545424461365, 0.8287749886512756 ] }
{ address: '/pos', args: [ 0.5364201068878174, 0.8287749886512756 ] }
{ address: '/pos', args: [ 0.5330424904823303, 0.8287749886512756 ] }
{ address: '/pos', args: [ 0.5266143083572388, 0.8287749886512756 ] }
{ address: '/pos', args: [ 0.5214642286300659, 0.8287749886512756 ] }
{ address: '/pos', args: [ 0.5214642286300659, 0.8285391330718994 ] }
{ address: '/pos', args: [ 0.5060290098190308, 0.8285391330718994 ] }
{ address: '/pos', args: [ 0.5060290098190308, 0.8252984285354614 ] }
{ address: '/pos', args: [ 0.46793174743652344, 0.8252984285354614 ] }
{ address: '/pos', args: [ 0.46793174743652344, 0.8096426725387573 ] }
{ address: '/pos', args: [ 0.45167502760887146, 0.8096426725387573 ]

In [7]:
processedLoops['firstLoop'][2]['pos']

{ x: 0.538375198841095, y: 0.8287749886512756 }